### This notebook dives deeper into questions answered during the peer review process. These results supported the improvement of the published work, but were not translated into figures for the main or supporting information text. 

In [1]:
import sys
from pathlib import Path
import os

ROOT_DIR = Path(os.getcwd()).resolve().parent
if (ROOT_DIR / "networks").exists():
    sys.path.append(str(ROOT_DIR))
else:
    raise FileNotFoundError("Root directory does not contain expected structure. Please adjust ROOT_DIR.")

In [2]:
# Standard library imports
import re
import pickle
import random
from itertools import zip_longest

# Data handling and numerical processing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Machine learning and data processing libraries
import torch
from scipy.stats import ttest_ind, stats
from tqdm import tqdm

# Custom utilities and feature processing
import src.utils as utils
from pycytominer.operations.transform import RobustMAD
from pycytominer import feature_select

/opt/miniconda3/envs/FeatureAggregation/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


# Load LINCS data

In [3]:
manualSeed = 42
print("Random Seed:", manualSeed)
random.seed(manualSeed)
np.random.seed(manualSeed)

encoding_label = 'moa'
mAP_label = 'Metadata_moa'
bestMOAs = pd.DataFrame()

DATASET_NAME = 'LINCS'
DOSE_POINT = 10
MAPfilename = f'mAP_{DATASET_NAME}_{DOSE_POINT}'

metadata_dir = str(ROOT_DIR) + '/aws_scripts/metadata/platemaps/2016_04_01_a549_48hr_batch1'

Random Seed: 42


In [4]:
repurposing_info = pd.read_csv(os.path.join(metadata_dir, 'repurposing_info_long.tsv'), index_col=False,
                               low_memory=False, sep='\t', usecols=["broad_id", "pert_iname", "moa"])
repurposing_info = repurposing_info.rename(columns={"broad_id": "broad_sample"})
repurposing_info = repurposing_info.drop_duplicates()
repurposing_info

,broad_sample,pert_iname,moa
0,BRD-K76022557-003-28-9,(R)-(-)-apomorphine,dopamine receptor agonist
16,BRD-K76022557-003-02-7,(R)-(-)-apomorphine,dopamine receptor agonist
32,BRD-K76022557-003-29-9,(R)-(-)-apomorphine,dopamine receptor agonist
48,BRD-K76022557-001-03-9,(R)-(-)-apomorphine,dopamine receptor agonist
64,BRD-K75516118-001-04-1,(R)-(-)-rolipram,phosphodiesterase inhibitor
...,...,...,...
39464,BRD-K00535541-001-06-3,9-aminoacridine,NaN
39465,BRD-K09291936-001-13-3,9-aminocamptothecin,topoisomerase inhibitor
39466,BRD-K09291936-001-14-9,9-aminocamptothecin,topoisomerase inhibitor
39467,BRD-K62607075-001-04-9,9-anthracenecarboxylic-acid,NaN


## Count number of unique MoAs

In [5]:
stain_platemap = str(ROOT_DIR) + r"/inputs/cpg0001_metadata/JUMP-MOA_compound_platemap_with_metadata.csv"
pm = pd.read_csv(stain_platemap)
print(len(pm.value_counts('moa')))
print(sum(pm.value_counts('moa')==4))

47
4


In [6]:
moa_counts = repurposing_info.groupby('moa')['pert_iname'].nunique()
moas_with_multiple_compounds = moa_counts[moa_counts >= 2]
number_of_moas = len(moas_with_multiple_compounds)

print("Number of unique MoAs linked to at least two unique compounds:", number_of_moas)
print("Minimum occurrence:", min(moas_with_multiple_compounds))
print("Maximum occurrence:", max(moas_with_multiple_compounds))
print("Median occurrence:", np.median(moas_with_multiple_compounds))

Number of unique MoAs linked to at least two unique compounds: 661
Minimum occurrence: 2
Maximum occurrence: 104
Median occurrence: 4.0


In [7]:
import re
file_path = str(ROOT_DIR) + "/aws_scripts/get_data_LINCS.txt"
pattern = r'SQ\d{8}'
platenames = []

with open(file_path, 'r') as file:
    for line in file:
        match = re.search(pattern, line)
        if match:
            platenames.append(match.group())

# Hack to remove either SQ00015203, SQ00015150, SQ00015151, SQ00015152, SQ00015153
# TODO test if it matters which one is removed
platenames.remove("SQ00015203")
print(platenames)
print(len(platenames))

['SQ00014812', 'SQ00014813', 'SQ00014814', 'SQ00014815', 'SQ00014816', 'SQ00014817', 'SQ00014818', 'SQ00014819', 'SQ00014820', 'SQ00015041', 'SQ00015042', 'SQ00015043', 'SQ00015044', 'SQ00015045', 'SQ00015046', 'SQ00015047', 'SQ00015048', 'SQ00015049', 'SQ00015050', 'SQ00015051', 'SQ00015052', 'SQ00015053', 'SQ00015054', 'SQ00015055', 'SQ00015056', 'SQ00015057', 'SQ00015058', 'SQ00015059', 'SQ00015096', 'SQ00015097', 'SQ00015098', 'SQ00015099', 'SQ00015100', 'SQ00015101', 'SQ00015102', 'SQ00015103', 'SQ00015105', 'SQ00015106', 'SQ00015107', 'SQ00015108', 'SQ00015109', 'SQ00015110', 'SQ00015111', 'SQ00015112', 'SQ00015116', 'SQ00015117', 'SQ00015118', 'SQ00015119', 'SQ00015120', 'SQ00015121', 'SQ00015122', 'SQ00015123', 'SQ00015124', 'SQ00015125', 'SQ00015126', 'SQ00015127', 'SQ00015128', 'SQ00015129', 'SQ00015130', 'SQ00015131', 'SQ00015132', 'SQ00015133', 'SQ00015134', 'SQ00015135', 'SQ00015136', 'SQ00015137', 'SQ00015138', 'SQ00015139', 'SQ00015140', 'SQ00015141', 'SQ00015142', 'SQ00

In [8]:
barcode_platemap = pd.read_csv(os.path.join(metadata_dir, 'barcode_platemap.csv'), index_col=False)
barcode_platemap = barcode_platemap[barcode_platemap['Assay_Plate_Barcode'].isin(platenames)]
barcode_platemap

,Assay_Plate_Barcode,Plate_Map_Name,Batch_Number,Batch_Date
0,SQ00015201,C-7161-01-LM6-017,1,2016-03-22
1,SQ00015202,C-7161-01-LM6-018,1,2016-03-22
2,SQ00015200,C-7161-01-LM6-016,1,2016-03-22
3,SQ00015204,C-7161-01-LM6-020,1,2016-03-22
4,SQ00015205,C-7161-01-LM6-021,1,2016-03-22
...,...,...,...,...
135,SQ00015149,C-7161-01-LM6-018,5,2016-05-30
136,SQ00015150,C-7161-01-LM6-019,5,2016-05-30
137,SQ00015151,C-7161-01-LM6-019,5,2016-05-30
138,SQ00015152,C-7161-01-LM6-019,5,2016-05-30


In [9]:
bigdf = []
for i, platestring in enumerate(platenames):
    # print('Getting data from: ' + platestring)
    platemap_name = barcode_platemap[barcode_platemap["Assay_Plate_Barcode"] == platestring]["Plate_Map_Name"].iloc[0]
    C_metadata = pd.read_csv(os.path.join(metadata_dir, 'platemap', platemap_name +'.txt'), sep='\t')
    if DOSE_POINT == 10:
        df = C_metadata[np.logical_and(C_metadata['mmoles_per_liter'] > 9, C_metadata['mmoles_per_liter'] < 11)]
    elif DOSE_POINT == 3:
        df = C_metadata[np.logical_and(C_metadata['mmoles_per_liter'] > 2.9, C_metadata['mmoles_per_liter'] < 6)]

    df['plate'] = platestring
    bigdf.append(df)

bigdf = pd.merge(pd.concat(bigdf), repurposing_info, on='broad_sample', how='left')
bigdf = utils.filterData(bigdf, 'negcon', encode='pert_iname', mode='LINCS')
shape1 = bigdf.shape[0]
bigdf.dropna(inplace=True)  # drop all compounds without annotations for pert_iname (and moa)
shape2 = bigdf.shape[0]
print("Removed", shape1-shape2, "wells due to missing annotation of pert_iname and moa.")
bigdf = bigdf[bigdf.Metadata_labels.duplicated(keep=False)]
print("Resulting df shape:", bigdf.shape)

/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/409212165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plate'] = platestring
/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/409212165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plate'] = platestring
/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/409212165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

Removed 1574 wells due to missing annotation of pert_iname and moa.
Resulting df shape: (5922, 10)


/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/409212165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plate'] = platestring
/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/409212165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plate'] = platestring
/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/409212165.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [10]:
# Fetch SQLIte file
# check which features it has and in which order 
# then remap the Cells_X features based on index number back to the original feature name 

In [15]:
lincs_mlp_profiles = pd.read_csv(str(ROOT_DIR) + "/inputs/cpg0004_profiles/mlp_profiles_10.csv")
lincs_average_profiles = pd.read_csv(str(ROOT_DIR) +  "/inputs/cpg0004_profiles/average_profiles_10.csv")
lincs_filtered_average_profiles = pd.read_csv(str(ROOT_DIR) +  "/inputs/cpg0004_profiles/filtered_average_profiles_10.csv")

In [16]:
temp_df = bigdf[['Metadata_labels', 'moa', 'pert_iname', 'plate']][~bigdf.Metadata_labels.duplicated(keep='last')]
temp_df = temp_df.rename(columns={'moa': 'Metadata_moa', 'pert_iname': 'Metadata_pert_iname', 'plate': 'Metadata_plate'})
lincs_mlp_profiles_ann = pd.merge(lincs_mlp_profiles, temp_df, on='Metadata_labels')
lincs_average_profiles_ann = pd.merge(lincs_average_profiles, temp_df, on='Metadata_labels')
lincs_filtered_average_profiles_ann = pd.merge(lincs_filtered_average_profiles, temp_df, on='Metadata_labels')

print('Dropping ', lincs_mlp_profiles_ann.shape[0] - lincs_mlp_profiles_ann.dropna().reset_index(drop=True).shape[0], 'rows due to NaNs')
lincs_mlp_profiles_ann = lincs_mlp_profiles_ann.dropna().reset_index(drop=True)
lincs_average_profiles_ann = lincs_average_profiles_ann.dropna().reset_index(drop=True)
lincs_filtered_average_profiles_ann = lincs_filtered_average_profiles_ann.dropna().reset_index(drop=True)
print('New size:', lincs_mlp_profiles_ann.shape)
shuffled_profiles = lincs_average_profiles_ann.copy()
shuffled_profiles.iloc[:, :-3] = pd.DataFrame.from_records(np.ones(shuffled_profiles.iloc[:, :-3].shape))

Dropping  0 rows due to NaNs
New size: (5922, 2052)


/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/318026203.py:13: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  shuffled_profiles.iloc[:, :-3] = pd.DataFrame.from_records(np.ones(shuffled_profiles.iloc[:, :-3].shape))


# Train Model on LINCS (requires loaded LINCS data)

In [17]:
NUM_WORKERS = os.cpu_count()

manualSeed = 42
print("Random Seed:", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
np.random.seed(manualSeed)

device = torch.device("cpu")
print("Device:", device)
print("Number of workers:", NUM_WORKERS)

save_name_extension = 'LINCS_aggregated_profiles'  # extension of the saved model, specify architecture used

model_name = 'model_' + save_name_extension
print(model_name)

lr = 0.00001
num_epochs = 75
batch_size = 256
input_dim = 595
kFilters = 1.0
latent_dim = 1024
output_dim = 1024
dropout = 0.0

Random Seed: 42
Device: cpu
Number of workers: 8
model_LINCS_aggregated_profiles


In [18]:
from torch.utils.data import TensorDataset, DataLoader
from networks.SimpleMLPs import MLPsumV2
from pytorch_metric_learning import losses, distances
import torch.optim as optim
from torch.utils.data import random_split

features = torch.tensor(lincs_average_profiles_ann[[col for col in lincs_average_profiles_ann.columns if 'Metadata' not in col]].to_numpy()).float()
features = torch.unsqueeze(features, dim=1)
pert_labels = torch.tensor(pd.Categorical(lincs_average_profiles_ann["Metadata_pert_iname"]).codes).long()

moa_labels = torch.tensor(pd.Categorical(lincs_average_profiles_ann["Metadata_moa"]).codes).long()
plate_names = lincs_average_profiles_ann["Metadata_plate"]

# %% Setup models
model = MLPsumV2(input_dim=input_dim, latent_dim=latent_dim, output_dim=output_dim,
                 k=kFilters, dropout=0, cell_layers=1,
                 proj_layers=2, reduction='sum')

# %% Setup optimizer
optimizer = optim.AdamW(model.parameters(), lr=lr)
loss_func = losses.SupConLoss(distance=distances.CosineSimilarity(), temperature=0.2) #[0.01 - 0.2]

# setup dataloader
dataset = TensorDataset(features, pert_labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# %% Start training
print(utils.now() + "Start training")
best_val = 0

epoch_tqdm = tqdm(range(num_epochs), leave=False, total=num_epochs)

for epoch in epoch_tqdm:
    # Training
    model.train()
    tr_loss = 0.0
    for idx, (data, labels) in enumerate(train_loader):
        feats, _ = model(data)

        # Loss
        tr_loss_tmp = loss_func(feats, labels)
        tr_loss += tr_loss_tmp.item()

        # Adam
        tr_loss_tmp.backward()
        optimizer.step()
        optimizer.zero_grad()

    tr_loss /= (idx+1)
    
    # Validation
    model.eval()
    val_loss = 0.0
    for idx, (data, labels) in enumerate(val_loader):
        feats, _ = model(data)

        # Loss
        val_loss_tmp = loss_func(feats, labels)
        val_loss += val_loss_tmp.item()

    val_loss /= (idx+1)
    
    with torch.no_grad():
        MLP_profiles = torch.tensor([], dtype=torch.float32)
        MLP_labels = torch.tensor([], dtype=torch.int16)
        for (data, labels) in val_loader:
            feats, _ = model(data)
            # Append everything to dataframes
            MLP_profiles = torch.cat([MLP_profiles, feats])
            MLP_labels = torch.cat([MLP_labels, labels])

        # Calculate mAP
        MLP_profiles = pd.concat(
            [pd.DataFrame(MLP_profiles.detach().numpy()), pd.Series(MLP_labels.detach().numpy())], axis=1)
        MLP_profiles.columns = [f"f{x}" for x in range(MLP_profiles.shape[1] - 1)] + ['Metadata_label']
        MLP_profiles = MLP_profiles[MLP_profiles.Metadata_label.duplicated(keep=False)]

        AP = utils.CalculateMAP(MLP_profiles, 'cosine_similarity',
                                groupby='Metadata_label', percent_matching=False)
        # assign the mAP (no running variable because this is calculated over the entire dataset)
        val_mAP = AP.AP.mean()

    epoch_summary = f"Epoch {epoch}. Train loss: {tr_loss}. Val loss: {val_loss}.  Val mAP: {val_mAP}"
    epoch_tqdm.set_description(utils.now() + epoch_summary)

    if val_mAP > best_val:
        best_val = val_mAP
        # print('Writing best val model checkpoint')
        # print('best val mAP:{}'.format(best_val))

        os.makedirs(os.path.join(str(ROOT_DIR), 'outputs'), exist_ok=True)
        torch.save(model.state_dict(), os.path.join(str(ROOT_DIR), 'outputs', f'model_bestval_{save_name_extension}'))

print('best val mAP:{}'.format(best_val))
print(utils.now() + 'Finished training')

2024-12-14 16:45:18.698155: Start training


best val mAP:0.5813129391561627
2024-12-14 16:46:06.971486: Finished training


In [19]:
counts = lincs_average_profiles_ann['Metadata_pert_iname'].value_counts()
more_than_five = counts[counts > 5].index

In [20]:
counts = lincs_average_profiles_ann['Metadata_moa'].value_counts()
more_than_five = counts[counts > 5].index

filtered_df = lincs_average_profiles_ann[lincs_average_profiles_ann['Metadata_moa'].isin(more_than_five)]
filtered_df

,Cells_3,Cells_4,Cells_5,Cells_7,Cells_15,Cells_18,Cells_19,Cells_20,Cells_21,Cells_22,...,Cells_1711,Cells_1713,Cells_1721,Cells_1725,Cells_1734,Cells_1743,Metadata_labels,Metadata_moa,Metadata_pert_iname,Metadata_plate
0,0.154495,0.056495,0.042508,-0.033523,-0.034429,-0.174949,0.044689,-0.235228,0.010372,0.120322,...,0.133481,-0.009466,0.066544,0.048834,-0.006535,0.046144,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208
1,0.062748,0.046352,0.013504,-0.018953,0.040173,-0.093455,0.052030,-0.110299,0.019014,0.178845,...,-0.118017,-0.006686,-0.109334,-0.102876,-0.147493,0.021132,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208
2,0.104607,0.038324,0.048719,-0.128391,0.008793,-0.121250,0.055463,-0.153951,0.008326,0.179112,...,-0.185395,-0.034891,0.001738,-0.091373,-0.068645,-0.119869,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208
3,0.207654,0.088107,-0.024833,-0.039286,-0.007888,-0.233223,0.032696,-0.264493,0.050855,0.107322,...,-0.129397,0.110213,0.018404,-0.514553,0.022239,0.058571,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208
4,0.041925,0.002196,0.032516,0.110651,0.018676,-0.023750,0.017621,-0.094685,-0.002828,0.082693,...,-0.015424,0.076927,-0.016742,-0.003475,-0.022121,0.070069,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5917,0.044248,0.201328,0.023369,0.565773,-0.037525,-0.219312,-0.007688,0.107773,0.035556,-0.156007,...,0.666168,0.072711,0.267628,0.342969,0.862694,-0.084763,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206
5918,-0.036413,0.141683,-0.029317,0.405021,-0.012749,-0.063697,-0.018915,0.210479,0.051571,-0.222081,...,-0.004686,0.061078,0.064806,0.037992,0.152769,0.037332,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206
5919,-0.192357,0.038504,-0.007814,0.808980,-0.013151,0.076645,0.007097,0.368720,-0.005047,-0.179842,...,0.321756,0.165887,-0.026189,0.144116,0.297226,0.255741,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206
5920,-0.100538,0.071190,0.026119,0.579744,-0.023511,-0.040675,-0.007331,0.260965,-0.069038,-0.190447,...,0.506538,0.076854,0.049784,0.215047,0.546934,0.018931,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206


In [21]:
# Average profile MoA
AP = utils.CalculateMAP(lincs_average_profiles_ann, 'cosine_similarity', groupby='Metadata_moa', percent_matching=True)
print(f"average profile MoA AP: {AP.AP.mean()}")
averageap = AP.AP.mean()
# Average profile Replicating
AP = utils.CalculateMAP(lincs_average_profiles_ann, 'cosine_similarity', groupby='Metadata_pert_iname', percent_matching=False)
print(f"average profile replicating AP: {AP.AP.mean()}")

average profile MoA AP: 0.03380586422617976
average profile replicating AP: 0.26899961118851373


In [23]:
trained_model = MLPsumV2(input_dim=input_dim, latent_dim=latent_dim, output_dim=output_dim,
                 k=kFilters, dropout=0, cell_layers=1,
                 proj_layers=2, reduction='sum')
trained_model.load_state_dict(torch.load(os.path.join(str(ROOT_DIR), 'outputs', f'model_bestval_{save_name_extension}')))
trained_model.eval()
with torch.no_grad():
    profiles, _ = trained_model(features)

    mlp_profiles = pd.DataFrame(profiles.numpy())
    mlp_profiles = pd.concat([mlp_profiles, pd.Series(moa_labels)], axis=1)
    mlp_profiles.columns = [f"f{x}" for x in range(mlp_profiles.shape[1] - 1)] + ['Metadata_label']
    mlp_profiles["Metadata_pert_iname"] = pert_labels

AP = utils.CalculateMAP(mlp_profiles, 'cosine_similarity', groupby='Metadata_label', percent_matching=True)
print(f"CSN profile MoA AP: {AP.AP.mean()}")
csnap = AP.AP.mean()
AP = utils.CalculateMAP(mlp_profiles, 'cosine_similarity', groupby='Metadata_pert_iname', percent_matching=False)
print(f"CSN profile replicating AP: {AP.AP.mean()}")

CSN profile MoA AP: 0.04398383583776106
CSN profile replicating AP: 0.3551148252730523


In [24]:
print(f"Improvement: {csnap - averageap}")

Improvement: 0.0101779716115813


### Quantify batch effects LINCS

In [26]:
from copairs.map import average_precision as copairs_ap

mlp_profiles = lincs_mlp_profiles_ann.dropna(axis=1)
meta = mlp_profiles[[col for col in mlp_profiles.columns if 'Metadata' in col]]
feats = mlp_profiles[[col for col in mlp_profiles.columns if 'Metadata' not in col]].to_numpy()
pos_sameby=['Metadata_plate']
pos_diffby=['Metadata_moa'] # if "Metadata_moa" is not multilabel, then we can included here too
neg_sameby=['Metadata_moa']
neg_diffby=["Metadata_plate"]

results = copairs_ap(
    meta, feats, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=20_000
)
results

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

,Metadata_labels,Metadata_moa,Metadata_pert_iname,Metadata_plate,n_pos_pairs,n_total_pairs,average_precision
0,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.935177
1,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.934244
2,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.942616
3,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.936680
4,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.938603
...,...,...,...,...,...,...,...
5917,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.957603
5918,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.953615
5919,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.954069
5920,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.953114


In [27]:
results.average_precision.mean()

0.914197711508827

In [28]:
average_profiles = lincs_filtered_average_profiles_ann.dropna(axis=1)
meta = average_profiles[[col for col in average_profiles.columns if 'Metadata' in col]]
feats = average_profiles[[col for col in average_profiles.columns if 'Metadata' not in col]].to_numpy()
pos_sameby=['Metadata_plate']
pos_diffby=['Metadata_moa'] # if "Metadata_moa" is not multilabel, then we can included here too
neg_sameby=['Metadata_moa']
neg_diffby=["Metadata_plate"]

results_average = copairs_ap(
    meta, feats, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=20_000
)
results_average

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

,Metadata_labels,Metadata_moa,Metadata_pert_iname,Metadata_plate,n_pos_pairs,n_total_pairs,average_precision
0,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.841551
1,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.805757
2,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.878886
3,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.822532
4,0,dopamine receptor agonist,(R)-(-)-apomorphine,SQ00015208,220,262,0.874565
...,...,...,...,...,...,...,...
5917,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.947607
5918,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.954188
5919,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.943068
5920,1228,P glycoprotein inhibitor,zosuquidar,SQ00015206,215,220,0.927933


In [29]:
results_average.average_precision.mean()

0.8970362647668788

# Load Stain data

In [32]:
rootdir = str(ROOT_DIR) + "/results/cpg0001/profiles"
plate1stain3 = pd.read_csv(rootdir + "/MLP_profiles_BR00115128.csv")
plate1stain3["Metadata_plate"] = "BR00115128"

plate1stain2 = pd.read_csv(rootdir + "/MLP_profiles_BR00112197binned.csv")
plate1stain2["Metadata_plate"] = "BR00112197binned"

plate2stain2 = pd.read_csv(rootdir + "/MLP_profiles_BR00113818.csv")
plate2stain2["Metadata_plate"] = "BR00113818"

plate3stain2 = pd.read_csv(rootdir + "/MLP_profiles_BR00112203.csv")
plate3stain2["Metadata_plate"] = "BR00112203"

plate4stain2 = pd.read_csv(rootdir + "/MLP_profiles_BR00112199.csv")
plate4stain2["Metadata_plate"] = "BR00112199"

In [33]:
mlp_profiles = pd.concat([plate1stain3, plate1stain2, plate2stain2, plate3stain2, plate4stain2])
mlp_profiles = pd.concat([plate1stain3, plate4stain2])

### Quantify batch effects Stain

In [34]:
from copairs.map import average_precision as copairs_ap

mlp_profiles = mlp_profiles.dropna(axis=1)
meta = mlp_profiles[[col for col in mlp_profiles.columns if 'Metadata' in col]]
feats = mlp_profiles[[col for col in mlp_profiles.columns if 'Metadata' not in col]].to_numpy()
pos_sameby=['Metadata_plate']
pos_diffby=['Metadata_labels'] # if "Metadata_moa" is not multilabel, then we can included here too
neg_sameby=['Metadata_labels']
neg_diffby=["Metadata_plate"]

results = copairs_ap(
    meta, feats, pos_sameby, pos_diffby, neg_sameby, neg_diffby, batch_size=20_000
)
results

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,Metadata_labels,Metadata_plate,n_pos_pairs,n_total_pairs,average_precision
0,8,BR00115128,356,360,0.989043
1,12,BR00115128,352,356,0.998080
2,29,BR00115128,352,356,0.996753
3,4,BR00115128,352,356,0.997846
4,38,BR00115128,352,356,0.973084
...,...,...,...,...,...
715,70,BR00112199,356,356,1.000000
716,56,BR00112199,356,356,1.000000
717,67,BR00112199,356,356,1.000000
718,54,BR00112199,356,356,1.000000


# MOA inspection

#### Make a list of the MOAs that are most (and least) improved by the method and then for each listing the top 3 CellProfiler features that distinguish that MOA from neg controls. Then add some commentary about that based on what we get.
Best MoAs (most improved from left to right)
JNK inhibitor	mTOR inhibitor,	inosine monophosphate dehydrogenase inhibitor,	LXR agonist,	IGF-1 inhibitor,	glycogen synthase kinase inhibitor,	p38 MAPK inhibitor,	p21 activated kinase inhibitor,	CHK inhibitor

Worst MoAs
MEK inhibitor,	phosphoinositide dependent kinase inhibitor,	DNA inhibitor,	bromodomain inhibitor,	histone lysine demethylase inhibitor,	DYRK inhibitor,	EGFR inhibitor,	BCL inhibitor,	hypoxia inducible factor inhibitor

#### ChatGPT response:
Looking at the list of best and worst MoAs, there doesn't seem to be an obvious biological pathway pattern that distinguishes between them. However, we can try to identify potential patterns based on the types of targets affected by these MoAs.

For example, some of the best MoAs (JNK inhibitor, mTOR inhibitor, p38 MAPK inhibitor) target kinases involved in signaling pathways regulating cell growth, proliferation, and survival. On the other hand, some of the worst MoAs (MEK inhibitor, EGFR inhibitor) also target kinases but may affect different downstream pathways or have different levels of redundancy in cellular signaling.

Similarly, some of the best MoAs (inosine monophosphate dehydrogenase inhibitor, glycogen synthase kinase inhibitor) may affect metabolic pathways, while some of the worst MoAs (histone lysine demethylase inhibitor, bromodomain inhibitor) target epigenetic regulators.

Overall, it seems that the model may be better at predicting outcomes for MoAs that affect specific signaling pathways or metabolic processes compared to those that target more diverse or complex mechanisms. However, further analysis would be needed to confirm these potential patterns and their biological significance.

In [45]:
plate1stain3 = pd.read_csv(rootdir + "/MLP_profiles_BR00115128.csv")
df = pd.read_csv(str(ROOT_DIR) + "/inputs/cpg0001_profiles/BR00116625_normalized_negcon.csv")
meta = df[[col for col in df.columns if 'Metadata' in col]]
metan = meta[meta["Metadata_control_type"] != "negcon"].reset_index(drop=True)
plate1stain3["Metadata_moa"] = metan["Metadata_moa"].copy()
plate1stain3["Metadata_pert_iname"] = metan["Metadata_pert_iname"].copy()

ap_mlp = utils.CalculateMAP(plate1stain3, 'cosine_similarity',
                        groupby="Metadata_moa", percent_matching=True)
ap_mlp[ap_mlp["compound"] == "hypoxia inducible factor inhibitor"]

,compound,AP,precision at R
224,hypoxia inducible factor inhibitor,-0.003791,0.0
225,hypoxia inducible factor inhibitor,-0.004169,0.0
226,hypoxia inducible factor inhibitor,-0.001442,0.0
227,hypoxia inducible factor inhibitor,-0.004064,0.0
228,hypoxia inducible factor inhibitor,-0.001887,0.0
229,hypoxia inducible factor inhibitor,-0.003786,0.0
230,hypoxia inducible factor inhibitor,-0.001178,0.0
231,hypoxia inducible factor inhibitor,-0.000887,0.0


In [46]:
top_moas = ["JNK inhibitor", "mTOR inhibitor", "inosine monophosphate dehydrogenase inhibitor"]
worst_moas = ["EGFR inhibitor", "BCL inhibitor", "hypoxia inducible factor inhibitor"] 
    
df = pd.read_csv("/Users/rdijk/Downloads/BR00116625_normalized_negcon.csv")
meta = df[[col for col in df.columns if 'Metadata' in col]]
feats = df[[col for col in df.columns if 'Metadata' not in col]].to_numpy()
negcon_feats = feats[meta["Metadata_control_type"] == "negcon"]
# Extract feature names from original dataframe
feature_names = [col for col in df.columns if 'Metadata' not in col]

top_feature_names = []
bot_feature_names = []
for top, bot in zip(top_moas, worst_moas):
    top_feats = feats[meta["Metadata_moa"] == top]
    bot_feats = feats[meta["Metadata_moa"] == bot]
    
    t_values_top = np.zeros(top_feats.shape[1])
    t_values_bot = np.zeros(bot_feats.shape[1])
    
    for i in range(top_feats.shape[1]):
        t_top, _ = ttest_ind(top_feats[:, i], negcon_feats[:, i])
        t_bot, _ = ttest_ind(bot_feats[:, i], negcon_feats[:, i])
        t_values_top[i] = abs(t_top)
        t_values_bot[i] = abs(t_bot)

    top_mask = ~np.isnan(t_values_top)
    bot_mask = ~np.isnan(t_values_top)
    t_values_top = t_values_top[top_mask]
    t_values_bot = t_values_bot[bot_mask]
    
    feature_names_topfilt = np.array(feature_names)[top_mask]
    feature_names_botfilt = np.array(feature_names)[bot_mask]

    # Get indices of top 5 features with largest absolute t-values
    top5_indices_top = np.argsort(t_values_top)[-5:]
    top5_indices_bot = np.argsort(t_values_bot)[-5:]
    
    for idx_top, idx_bot in zip(top5_indices_top, top5_indices_bot):
        top_feature_names.append(feature_names_topfilt[idx_top])
        bot_feature_names.append(feature_names_botfilt[idx_bot])

# top_feature_names = set(top_feature_names)
# bot_feature_names = set(bot_feature_names)
st1 = "Top MoA feature names"
st2 = "Bottom MoA feature names"
print(f"{st1.ljust(70)} {st2}")
# for top_feat, bot_feat in zip_longest(top_feature_names, bot_feature_names, fillvalue=""):
#     print(f"{top_feat.ljust(70)} {bot_feat}")

for f in bot_feature_names:
    print(f)

/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/3768380282.py:22: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_bot, _ = ttest_ind(bot_feats[:, i], negcon_feats[:, i])
/var/folders/0n/sy07p4_n4jzgg1n5phyn9yq00000gn/T/ipykernel_87730/3768380282.py:21: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_top, _ = ttest_ind(top_feats[:, i], negcon_feats[:, i])


Top MoA feature names                                                  Bottom MoA feature names
Cytoplasm_Correlation_Costes_RNA_Brightfield
Nuclei_Correlation_Manders_Brightfield_AGP
Nuclei_Correlation_Manders_DNA_AGP
Cells_Correlation_Costes_AGP_Brightfield
Nuclei_Correlation_Costes_Brightfield_AGP
Cytoplasm_Correlation_Costes_ER_Brightfield
Cells_Correlation_Costes_Brightfield_ER
Nuclei_Correlation_Costes_DNA_Brightfield
Cells_Correlation_Costes_Brightfield_Mito
Nuclei_Correlation_Costes_Brightfield_DNA
Cells_Texture_InfoMeas2_ER_20_01
Cytoplasm_Texture_InfoMeas2_ER_20_00
Cytoplasm_Texture_InfoMeas2_ER_20_02
Cells_Texture_InfoMeas2_ER_20_03
Nuclei_Correlation_Manders_Brightfield_AGP


In [47]:
jt = df.Metadata_moa.value_counts()
sum(jt>4)

43

In [48]:
# subdf = df[(df["Metadata_moa"] == "BCL inhibitor") | (df["Metadata_control_type"] == "negcon")].copy()
subdf = df.copy()

features = subdf[[c for c in subdf.columns if not c.startswith("Metadata")]]

scaler = RobustMAD(epsilon=0)
fitted_scaler = scaler.fit(features)
features = fitted_scaler.transform(features)
features = features.dropna(axis=1, how='any')
features = feature_select(features, operation=["variance_threshold", "correlation_threshold",
                                               "drop_na_columns", "blocklist"])

features['Metadata_labels'] = subdf['Metadata_pert_iname']
features['Metadata_pert_iname'] = subdf['Metadata_pert_iname']
features = features[subdf["Metadata_control_type"] != 'negcon']

apdf = utils.CalculateMAP(features, 'cosine_similarity', 'Metadata_labels', False)
apdf['Metadata_moa'] = subdf[subdf["Metadata_control_type"] != 'negcon']['Metadata_moa']

for i in range(len(top_moas)):
    print(f"{top_moas[i]}: {apdf[apdf.Metadata_moa == top_moas[i]]['AP'].mean()}")

for i in range(len(worst_moas)):
    print(f"{worst_moas[i]}: {apdf[apdf.Metadata_moa == worst_moas[i]]['AP'].mean()}")

JNK inhibitor: 0.48118817878911413
mTOR inhibitor: 0.4029949606181567
inosine monophosphate dehydrogenase inhibitor: 0.02900384086346854
EGFR inhibitor: 0.3329800717661998
BCL inhibitor: 0.12818878782478393
hypoxia inducible factor inhibitor: 0.38526914975492327


In [38]:
apdf.fillna('str').groupby("compound").mean() # EGFR inhibitor vs negcon 

,AP,precision at R
compound,,
CP-724714,0.262742,0.250000
neratinib,0.903226,1.000000
str,0.138505,0.817029


In [40]:
apdf.fillna('str').groupby("compound").mean() # hypoxia inducible factor inhibitor vs negcon 

,AP,precision at R
compound,,
IOX2,0.390118,0.50000
acriflavine,0.903226,1.00000
str,0.079973,0.76087


In [42]:
apdf.fillna('str').groupby("compound").mean() # BCL inhibitor vs negcon

,AP,precision at R
compound,,
ABT-737,0.903226,1.00000
Compound3,0.275713,0.25000
str,0.050233,0.76087


# mAP replicating and MoA retrieval for sanity checks

In [ ]:
## Calculate mean average precision
ap_mlp = utils.CalculateMAP(lincs_mlp_profiles_ann, 'cosine_similarity',
                        groupby=mAP_label, percent_matching=True)
ap_bm = utils.CalculateMAP(average_profiles_ann, 'cosine_similarity',
                        groupby=mAP_label, percent_matching=True)
ap_bm_filtered = utils.CalculateMAP(filtered_average_profiles_ann, 'cosine_similarity',
                        groupby=mAP_label, percent_matching=True)
ap_shuffled = utils.CalculateMAP(shuffled_profiles, 'cosine_similarity',
                        groupby=mAP_label, percent_matching=True)

ap_mlp['compound'] = ap_mlp.compound.str.replace('|', '/')
ap_bm['compound'] = ap_bm.compound.str.replace('|', '/')
ap_bm_filtered['compound'] = ap_bm_filtered.compound.str.replace('|', '/')


In [13]:
ap_mlp['count'] = [1]*len(ap_mlp)
ap_bm['count'] = [1]*len(ap_bm)
ap_bm_filtered['count'] = [1]*len(ap_bm_filtered)

ap_mlp.groupby('compound').agg({'AP': 'mean', 'count': 'sum'})
ap_bm.groupby('compound').agg({'AP': 'mean', 'count': 'sum'})
ap_bm_filtered.groupby('compound').agg({'AP': 'mean', 'count': 'sum'})

print('Total mean mAP MLP:', ap_mlp.AP.mean(), '\nTotal mean precision at R MLP:', ap_mlp['precision at R'].mean())
# print(ap_mlp.groupby('compound').mean().sort_values('AP').iloc[-30:, :].round(4).to_markdown())
print('\n')
print('Total mean mAP BM:', ap_bm.AP.mean(), '\nTotal mean precision at R BM:', ap_bm['precision at R'].mean())
# print(ap_bm.groupby('compound').mean().sort_values('AP').iloc[-30:, :].round(4).to_markdown())
print('\n')
print('Total mean mAP filtered BM:', ap_bm_filtered.AP.mean(), '\nTotal mean precision at R BM:', ap_bm_filtered['precision at R'].mean())
# print(ap_bm_filtered.groupby('compound').mean().sort_values('AP').iloc[-30:, :].round(4).to_markdown())
print('\n')
print('Total mean mAP shuffled:', ap_shuffled.AP.mean(), '\nTotal mean precision at R shuffled:', ap_shuffled['precision at R'].mean())

print('\n')
# Conduct Welch's t-Test and print the result
print("Welch's t-test between mlp mAP and bm mAP:", stats.ttest_ind(np.array(ap_mlp.AP), np.array(ap_bm.AP), equal_var=False))
print('\n')

Total mean mAP MLP: 0.05428360457109933 
Total mean precision at R MLP: 0.05923119438247785


Total mean mAP BM: 0.0340071826597065 
Total mean precision at R BM: 0.04146744649885223


Total mean mAP filtered BM: 0.03400078109097556 
Total mean precision at R BM: 0.04151388441260053


Total mean mAP shuffled: 0.0 
Total mean precision at R shuffled: 0.006677718234667885


Welch's t-test between mlp mAP and bm mAP: TtestResult(statistic=6.742721637471297, pvalue=1.6730198014776555e-11, df=7307.3689208088335)


